# Term frequency x Inverse document frequency - TfIdf

With acknowledgement to Mayank Tripathi https://github.com/mayank408

We will build on the BoW model creating a TfIdf model from first principles.

First a couple imports: "string" to do some string manipulation, pprint and pandas to help us print our data structures.

In [0]:
import string
import pprint as pp

import pandas as pd



Now our corpus:

In [0]:
documents = ['Klonopin 0.25 mg po every evening, Fluconazole 200 mg po daily, Synthroid 125 mcg po every day',
             'she will not consider switching to clozapine',
             'lovastatin 40 mg one half tab po daily, multivitamin daily, metformin 500 mg one tab po twice a day',
             'Aspirin 81 mg po once daily, Zoloft 25 mg po once daily, Calcium with vitamin D two tablets po once daily']


We will "normalise" our documents, to lower case them and remove punctuation.

In [0]:
normalised_documents = []
for i in documents:
    no_punctuation = ''.join(c for c in i if c not in string.punctuation)
    normalised_documents.append(no_punctuation.lower())
    
for i in normalised_documents:
  print(i)

Let's split in to tokens, to give our "bags"

In [0]:
bows = []
for i in normalised_documents:
    bows.append(i.split(' '))

print(bows)

We need to get a set containing all of our unique words, so that we can calculate their relative 
frequencies in each document and across all documents.

In [0]:
word_set = set()
for i in bows:
  word_set = word_set.union(set(i))
print(word_set)

Let's count how many of each word we have for each of our bags:

In [0]:
wordCounts = []
for i in bows:
  thisWordCount = dict.fromkeys(word_set, 0)
  for word in i:
    thisWordCount[word]+=1
  wordCounts.append(thisWordCount)
 

Let's take a look at these counts:

In [0]:
pd.DataFrame(wordCounts)

Now we will define ***Term Frequency*** (TF) as the relative frequency of a word in a bag (document) - i.e. what fraction of all words in a document is a particular word? We will define a function to compute this for all of the words in a bag.

In [0]:
def computeTF(wordCount, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordCount.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

We will run this function over all of our bags (documents), and put the resulting TFs in a single data structure. Tale a look and see how documents differ, and how the TFs reflect relative occurence of a word in each document.

In [0]:
termFreqs = []
for i in range(0,len(bows)): 
  termFreqs.append(computeTF(wordCounts[i],bows[i]))

pd.DataFrame(termFreqs) 

Our next function defines ***Inverse Docuemnt Frequency*** - IDF. This measures the rareness of a word across our whole collection of documents. For each word, we divide the total number of documents by the number containing that word. We take the log of this. 

In [0]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

Now we compute IDF for our words. Take a look at the difference between common words like "mg" and rare ones like drug names.

In [0]:
idfs = computeIDF(wordCounts)
pp.pprint(idfs)

What do the IDFs look like?

In [ ]:
pd.Series(idfs).sort_index(ascending=False).plot.barh(figsize=(10,10))

Can you explain the range of values that IDF takes, and the maximum value?

Let's define a function to put TF and IDF together.

In [0]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

And now run this over all of the documents in our term frequency list:

In [0]:
tfidfs = []
for i in termFreqs:
  tfidfs.append(computeTFIDF(i, idfs))
  

  
pd.DataFrame(tfidfs)


How do these compare to the term frequencies? Run the next line to get just the TFs. What differences are there?

In [0]:
pd.DataFrame(termFreqs)